In [120]:
import GEOparse
import pandas as pd
from unipressed import IdMappingClient, UniprotkbClient
import time
from tqdm import tqdm
tqdm.pandas()
from Bio import Entrez

In [121]:
gse = GEOparse.get_GEO("GSE12345")

28-Sep-2024 00:34:28 DEBUG utils - Directory ./ already exists. Skipping.
28-Sep-2024 00:34:28 INFO GEOparse - File already exist: using local version.
28-Sep-2024 00:34:28 INFO GEOparse - Parsing ./GSE12345_family.soft.gz: 
28-Sep-2024 00:34:28 DEBUG GEOparse - DATABASE: GeoMiame
28-Sep-2024 00:34:28 DEBUG GEOparse - SERIES: GSE12345
28-Sep-2024 00:34:28 DEBUG GEOparse - PLATFORM: GPL570
/home/olivier/micromamba/envs/hskp_test/lib/python3.11/site-packages/GEOparse/GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
28-Sep-2024 00:34:29 DEBUG GEOparse - SAMPLE: GSM309986
28-Sep-2024 00:34:29 DEBUG GEOparse - SAMPLE: GSM309987
28-Sep-2024 00:34:29 DEBUG GEOparse - SAMPLE: GSM309988
28-Sep-2024 00:34:30 DEBUG GEOparse - SAMPLE: GSM309989
28-Sep-2024 00:34:30 DEBUG GEOparse - SAMPLE: GSM309990
28-Sep-2024 00:34:30 DEBUG GEOparse - SAMPLE: GSM309991
28-Sep-2024 00:34

In [122]:
# Concatenate expression data from all samples into a single DataFrame
gsm_dfs = []
for gsm_name, gsm in gse.gsms.items():
    gsm_df = gsm.table
    gsm_df = gsm_df.set_index('ID_REF').rename(columns={'VALUE': gsm_name})
    gsm_dfs.append(gsm_df)
expression_df = pd.concat(gsm_dfs, axis=1)

In [124]:
# getting platform data (there can be multiple ones)
descriptive_columns_to_uniprot_source = {
    'GB_ACC': 'EMBL-GenBank-DDBJ', 
    #'Gene Symbol': 'Gene_Name', 
    'ENTREZ_GENE_ID': 'GeneID', 
    'RefSeq Transcript ID': 'RefSeq_Nucleotide'
}
gpl_dfs = []
for gpl in gse.gpls.values():
    
    if gpl.table.empty:
        continue
        
    descr_column_found = False
    for descr_column in descriptive_columns_to_uniprot_source.keys():
        if descr_column in gpl.table.columns:
            #cols_to_keep = ['ID', descr_column, 'Species Scientific Name']
            cols_to_keep = ['ID', descr_column]
            gpl_df = gpl.table[cols_to_keep]
            gpl_dfs.append(gpl_df)
            descr_column_found = True
            break
    
    if not descr_column_found:
        raise KeyError(f'Could not find a descriptive column in the GPL table. Found columns: {gpl.table.columns.to_list()}')
            
mapping_df = pd.concat(gpl_dfs, ignore_index=True).set_index('ID')

In [126]:
"""
# getting species name
species_names = mapping_df['Species Scientific Name'].unique().tolist()
if len(species_names) > 1:
    raise ValueError('More than one species')

species_name = species_names[0]
mapping_df.drop(columns=['Species Scientific Name'], inplace=True)
"""

In [127]:
def get_uniprot_ids(ids: list[str], source: str) -> dict:    
    request = IdMappingClient.submit(source=source, dest="UniProtKB", ids=ids)
    while request.get_status() != 'FINISHED':
        time.sleep(1)
    return {result_dict['from'] : result_dict['to'] for result_dict in request.each_result()}

In [128]:
def chunk_list(lst: list, chunksize: int):
    return [lst[i: i + chunksize] for i in range(0, len(lst), chunksize)]

In [ ]:
mapping_df = mapping_df[:1000]

In [130]:
for i, col in enumerate(descriptive_columns_to_uniprot_source):
    
    if col in mapping_df.columns:
        
        # unique list of IDs for this type of ID
        ids = mapping_df[col].dropna().unique().tolist()
        
        if ids:
            source = descriptive_columns_to_uniprot_source[col]
            chunks = chunk_list(ids, chunksize=2000)
            mappings = {}
            for chunk in tqdm(chunks):
                # converting to uniprot IDs for all IDs comprised in this chunk
                mapping = get_uniprot_ids(chunk, source)
                mappings.update(mapping)
                
            # making a series out of this dict: index is previous ids while value is uniprot id
            uniprot_mapping_series = pd.Series(mappings, name=f'uniprot_id_{i}')
            # left join to get a new uniprot_id_{i} col where values are not NA where mapping[col].notna()
            mapping_df = mapping_df.merge(uniprot_mapping_series, how='left', left_on=col, right_index=True)

#getting list of uniprot id columns in dataframe
uniprot_columns = [col for col in mapping_df.columns if col.startswith('uniprot_id_')]
# coalescing all uniprot ids into a single uniprot_id column
mapping_df['uniprot_id'] = mapping_df[uniprot_columns].bfill(axis=1).iloc[:, 0]
mapping_df.drop(columns=uniprot_columns, inplace=True)
    

100%|██████████| 1/1 [00:00<00:00,  1.49it/s]
/tmp/ipykernel_4163/3745249323.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mapping_df['uniprot_id'] = mapping_df[uniprot_columns].bfill(axis=1).iloc[:, 0]
